# 02. Build SQLite Database (CoffeeKing)

**Goal:** Create a SQLite databae from Yelp JSON files so we can run SQL queries (joins, aggregations) for Milestone 3-4.

**Tables (initial):**
- `coffee_business` (from business.json, coffee-related subset only)
- `coffee_reviews` (from review.json, only reviews for coffee_businesss)

> Note: Raw JSON files are not committed to GitHub. The SQLite DB file is also not committed.

In [1]:
import pandas as pd
import sqlite3
from pathlib import Path

DATA_RAW = Path("../data_raw")
DB_PATH = Path("../db/coffeeking.db")

biz_path = DATA_RAW / "yelp_academic_dataset_business.json"
review_path = DATA_RAW / "yelp_academic_dataset_review.json"

conn = sqlite3.connect(DB_PATH)
print("DB connected:", DB_PATH.resolve())

DB connected: /Users/kwaknakyung/projects/Coffeeking-Yelp/db/coffeeking.db


## 2. Build `coffee_business` table (from business.json)

We filter coffee-related businesses using a simple keyword match on `categories`.
This keeps the DB light and focused for SQL analysis.

In [2]:
biz = pd.read_json(biz_path, lines= True)

biz["categories_clean"] = biz["categories"].fillna("").str.lower()
coffee_keywords = ["coffee", "cafe", "cafes", "coffee & tea"]

coffee_mask = biz["categories_clean"].str.contains("|".join(coffee_keywords), regex=True)
coffee_biz = biz.loc[coffee_mask].copy()

coffee_biz.shape

(8509, 15)

In [3]:
coffee_business = coffee_biz [
    ["business_id", "name", "address", "city", "state", "postal_code",
    "latitude", "longitude", "stars", "review_count", "is_open", "categories"]
].copy()

coffee_business.head(3)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
20,WKMJwqnfZKsAae75RMP6jA,Roast Coffeehouse and Wine Bar,10359 104 Street NW,Edmonton,AB,T5J 1B9,53.546045,-113.499169,4.0,40,0,"Coffee & Tea, Food, Cafes, Bars, Wine Bars, Re..."
46,JX4tUpd09YFchLBuI43lGw,Naked Cyber Cafe & Espresso Bar,10303 108 Street NW,Edmonton,AB,T5J 1L7,53.544682,-113.506589,4.0,12,1,"Arts & Entertainment, Music Venues, Internet S..."


In [4]:
coffee_business.to_sql("coffee_business", conn, if_exists="replace", index=False)
print("Saved table: coffee_business")

Saved table: coffee_business


In [5]:
pd.read_sql_query("SELECT COUNT(*) AS n_rows FROM coffee_business;", conn)

,n_rows
0,8509


## 3. Build `coffee_reviews` table (from review.json, filtered)

`review.json` is very large, so we read it in chunks.
We keep only reviews whose `business_id` exists in `coffee_business`.

In [6]:
# pull business_ids directly from SQLite (so DB is the source of truth)
coffee_ids_df = pd.read_sql_query("SELECT business_id FROM coffee_business;", conn)
coffee_ids = set(coffee_ids_df["business_id"])

len(coffee_ids)

8509

In [7]:
chunk_size = 50000
max_chunks = 2

kept = 0
chunks_done = 0

# start fresh for reproducibility
conn.execute("DROP TABLE IF EXISTS coffee_reviews;")
conn.commit()

for chunk in pd.read_json(review_path, lines=True, chunksize=chunk_size):
    # filter to only coffee business_ids
    chunk = chunk[chunk["business_id"].isin(coffee_ids)]

    if not chunk.empty:
        chunk_small = chunk[["review_id", "business_id", "user_id", "stars", "date", "text"]].copy()
        chunk_small.to_sql("coffee_reviews", conn, if_exists="append", index = False)
        kept += len(chunk_small)

    chunks_done += 1
    print(f"chunk {chunks_done} done | kept so far: {kept}")

    if chunks_done >= max_chunks:
        break

kept

chunk 1 done | kept so far: 5670
chunk 2 done | kept so far: 11351


11351

In [8]:
pd.read_sql_query("SELECT COUNT(*) AS n_rows FROM coffee_reviews;", conn)

,n_rows
0,11351


## 4. Sanity check: join coffee_business + coffee_reviews

Goal: Verify the join works and that reviews aggregate correctly by geography.

In [10]:
q = """
SELECT
    b.state,
    COUNT(DISTINCT b.business_id) AS n_business,
    COUNT(r.review_id) AS n_reviews,
    ROUND(AVG(r.stars), 3) AS avg_review_stars
FROM coffee_business b
LEFT JOIN coffee_reviews r
ON b.business_id = r.business_id
GROUP BY b.state
ORDER BY n_reviews DESC
LIMIT 10;
"""
pd.read_sql_query(q, conn)

,state,n_business,n_reviews,avg_review_stars
0,LA,633,2554,4.116
1,PA,2128,2084,3.918
2,FL,1519,1749,4.053
3,TN,647,1046,4.016
4,IN,573,910,4.057
5,MO,551,681,3.783
6,CA,243,587,4.083
7,NV,327,556,4.144
8,NJ,487,389,3.866
9,AZ,435,374,3.684


In [11]:
q2 = """
SELECT
    SUM(CASE WHEN r.n_reviews IS NULL THEN 1 ELSE 0 END) AS business_with_0_reviews,
    COUNT(*) AS total_businesses
FROM(
    SELECT business_id FROM coffee_business
) b
LEFT JOIN (
    SELECT business_id, COUNT(*) AS n_reviews
    FROM coffee_reviews
    GROUP BY business_id
) r
ON b.business_id = r.business_id;
"""
pd.read_sql_query(q2, conn)

,business_with_0_reviews,total_businesses
0,7876,8509


### Quick takeaways (sanity check)
- Join works: `coffee_business` <=> `coffee_reviews` by `business_id`.
- Current review coverage is very low because we only loaded 2 chunks of `review.json`.
    - Businesses with 0 loaded reviews: **7,876 / 8,509** (92.6%)
- Next: fully load `coffee_reviews` from `review.json` so review-based insights are representative.


## 5. Full load `coffee_reviews` (all matching reviews)

We rebuild `coffee_reviews`from scratch and load all reviews for coffee-related businesses.
This is required before any reliable review-based statistics or text analysis.

In [12]:
import time
import pandas as pd

review_path = "../data_raw/yelp_academic_dataset_review.json"
chunk_size = 50000

# coffee_ids must exist
# coffee_ids = set(coffee_biz["business_id"])

conn.execute("DROP TABLE IF EXISTS coffee_reviews;")
conn.commit()

kept = 0
chunks_done = 0
t0 = time.time()

for chunk in pd.read_json(review_path, lines=True, chunksize=chunk_size):
    chunk = chunk[chunk["business_id"].isin(coffee_ids)]
    if not chunk.empty:
        chunk_small = chunk[["review_id", "business_id", "user_id", "stars", "date", "text"]].copy()
        chunk_small.to_sql("coffee_reviews", conn, if_exists = "append", index = False)
        kept += len(chunk_small)

    chunks_done += 1
    if chunks_done % 10 == 0:
        elapsed = time.time() - t0
        print(f"chunks: {chunks_done} | kept: {kept} | elapsed: {elapsed:.1f}s")

kept

chunks: 10 | kept: 55502 | elapsed: 4.1s
chunks: 20 | kept: 102670 | elapsed: 8.2s
chunks: 30 | kept: 146450 | elapsed: 12.1s
chunks: 40 | kept: 189507 | elapsed: 16.0s
chunks: 50 | kept: 230068 | elapsed: 19.9s
chunks: 60 | kept: 271705 | elapsed: 23.8s
chunks: 70 | kept: 315770 | elapsed: 29.7s
chunks: 80 | kept: 358319 | elapsed: 35.2s
chunks: 90 | kept: 398562 | elapsed: 39.8s
chunks: 100 | kept: 442231 | elapsed: 44.2s
chunks: 110 | kept: 487771 | elapsed: 48.8s
chunks: 120 | kept: 535104 | elapsed: 53.1s
chunks: 130 | kept: 584129 | elapsed: 57.4s
chunks: 140 | kept: 630349 | elapsed: 61.7s


630349

In [13]:
pd.read_sql_query("SELECT COUNT(*) AS n_rows FROM coffee_reviews;", conn)

,n_rows
0,630349


In [14]:
q = """
SELECT
    SUM(CASE WHEN r.n_reviews IS NULL THEN 1 ELSE 0 END) AS business_with_0_reviews,
    COUNT(*) AS total_businesses
FROM coffee_business b
LEFT JOIN(
    SELECT business_id, COUNT(*) AS n_reviews
    FROM coffee_reviews
    GROUP BY business_id
) r
ON b.business_id = r.business_id;
"""
pd.read_sql_query(q, conn)

,business_with_0_reviews,total_businesses
0,0,8509


## 6. Correlations (ratings vs popularity)

Now that all coffee-related reviews are loaded, we can study relationship such as:

- Are higher-rated coffee businesses also more "popular" (have more reviews)?
- Do some states/cities have systematically higher ratings or higher review volume?

We will compute:
1) Business-level average review stars from `coffee_reviews`
2) Correlation between review volume and average rating

In [15]:
q = """
SELECT
    b.business_id,
    b.name,
    b.city,
    b.state,
    b.stars AS business_stars, --Yelp business table rating
    b.review_count AS business_review_count, --Yelp business table count
    COUNT(r.review_id) AS n_reviews_loaded, --from review table
    AVG(r.stars) AS avg_review_stars
FROM coffee_business b
JOIN coffee_reviews r
ON b.business_id = r.business_id
GROUP BY b.business_id, b.name, b.city, b.state, b.stars, b.review_count;
"""
biz_summary = pd.read_sql_query(q, conn)
biz_summary.head()

,business_id,name,city,state,business_stars,business_review_count,n_reviews_loaded,avg_review_stars
0,-0__F9fnKt8uioCKztF5Ww,Piscobar,New Orleans,LA,4.0,66,66,3.878788
1,-0eUa8TsXFFy0FCxHYmrjg,Waterfront Gourmet Cafe & Deli,Philadelphia,PA,4.0,26,28,3.964286
2,-0epFLgYq2C1Jo_W4FOBKw,Our House Cafe,Columbia,IL,4.0,15,18,4.222222
3,-0gRYq5UjMtZbELj0KHxzA,La Colombe Coffee,Bryn Mawr,PA,3.5,128,133,3.593985
4,-0iIxySkp97WNlwK66OGWg,Truckee Bagel Company - Midtown,Reno,NV,3.5,219,233,3.721030


In [16]:
biz_summary.shape, biz_summary[["n_reviews_loaded", "avg_review_stars"]].describe()

((8509, 8),
        n_reviews_loaded  avg_review_stars
 count       8509.000000       8509.000000
 mean          74.080268          3.592373
 std          174.002102          0.951368
 min            5.000000          1.000000
 25%           12.000000          3.000000
 50%           27.000000          3.833333
 75%           69.000000          4.339286
 max         5778.000000          5.000000)

In [17]:
corr = biz_summary[["n_reviews_loaded", "avg_review_stars"]].corr(numeric_only=True)
corr

,n_reviews_loaded,avg_review_stars
n_reviews_loaded,1.000000,0.153095
avg_review_stars,0.153095,1.000000


### Interpretation (Correlation)

- Correlation between **n_reviews_loaded** (popularity proxy) and **avg_review_stars** (quality proxy) is **0.153**.
- This is a **weak positive relationship**, meaning: businesses with more reviews are only *slightly* more likely to have average rating.
- Takeaway for CoffeeKing: popularity and quality should be treated as **separate signals** (don't assume "busy" = "best")

## 7. New Metric: Visibility Score (Popularity x Quality)

Review counts are extremely skewed (a few places have thousands of reviews), so we log-transform review volume.
Then we combine it with average review stars to capture "well-known AND well-liked" businesses.

**Visibility Score = log(1 + n_reviews_loaded) x avg_review_stars**

Why this helps CoffeeKing:
- Helps identify benchmark businessses that are both popular and consistently rated well.
- Produces a ranking that is less distorted by extreme outliers than raw review counts. 

In [18]:
import numpy as np

biz_summary["visibility_score"] = np.log1p(biz_summary["n_reviews_loaded"]) * biz_summary["avg_review_stars"]

biz_summary.sort_values("visibility_score", ascending=False)[
    ["name", "city", "state", "n_reviews_loaded", "avg_review_stars", "visibility_score"]
].head(15)

,name,city,state,n_reviews_loaded,avg_review_stars,visibility_score
8356,Reading Terminal Market,Philadelphia,PA,5778,4.605400,39.891908
6903,Ruby Slipper - New Orleans,New Orleans,LA,5264,4.291223,36.770791
482,Mazzaro's Italian Market,Saint Petersburg,FL,1583,4.641188,34.194918
5725,Bodega,Saint Petersburg,FL,1812,4.556291,34.184662
7872,Surrey's Café & Juice Bar,New Orleans,LA,2120,4.460849,34.168511
2849,Cafe La Maude,Philadelphia,PA,1530,4.633333,33.979367
1384,District Donuts Sliders Brew,New Orleans,LA,2110,4.378199,33.514751
6208,Parc,Philadelphia,PA,2884,4.146671,33.037692
6045,Willa Jean,New Orleans,LA,2050,4.246829,32.386671
7095,Oxford Exchange,Tampa,FL,1938,4.270898,32.330388


In [19]:
(biz_summary["business_review_count"] - biz_summary["n_reviews_loaded"]).describe()

count    8509.000000
mean       -2.404513
std         5.193207
min      -123.000000
25%        -3.000000
50%        -1.000000
75%         0.000000
max         0.000000
dtype: float64

### Key Findings: "Visibility Score" Top Businesses

Top-ranked businesses by **Visibility Score = log(1+n_reviews_loaded) x avg_review_stars** are heavily concentrated in a few cities/states:

- **Philadelphia, PA** and **New Orleans, LA** appear repeatedly in the top list.
- **Florida (Saint Petersburg, Tampa)** also shows multiple high-visibility coffee-related destinations.

Interpretation:
- The most "visible" coffee-related listings often look like **coffee + destination food** hybrids (markets, brunch, donuts, etc.), not only small standalone coffee shops.
- This suggests CoffeeKing should separate strategy into:
(1) "destination/ food-forward café" positioning vs.
(2) "pure coffee shop" positioning,
because the success drivers (review_volume and customer expectations) may differ.

### Data sanity check: business review_count vs loaded reviews

We compared `busienss_review_count` (from business.json) with `n_reviews_loaded` (counted from review.json).

Result (business_review_count - n_reviews_loaded):
- Median difference is around **-1** and **75% of businesses have a difference of 0**.
- The minimum difference reaches **-123**, meaning a small number of businesses have slightly more reviews in review.json than the business.json review_count filed.

Takeaway:
- Overall, our review loading/join looks consistent.
- Small mismatches are expected in large public datasets and do not impact high-level insights.

## 8. Concept Tagging (Positioning) - Pure Coffee vs Food-forward

Goal: Create simple, explainable "concept tags" from the Yelp `categories` field so we can compare business performance by positioning.
This makes the project more actionable for CoffeeKing: it's not only "where to open" but also "what concept mix tends to win."

In [21]:
# Build concept tags in pandas (safe + explainable rules)

import re
import numpy as np

# normalize categories text
coffee_biz["categories_clean"] = coffee_biz["categories"].fillna("").str.lower()

# define keyword buckets (simple on purpose)
pure_coffee_kw = r"(coffee|coffee\s*&\s*tea|cafe|cafes|espresso|tea|bubble tea)"
food_kw = r"(breakfast|brunch|bakery|bakeries|donut|donuts|dessert|ice cream|sandwich|deli|restaurant|restaurants|pizza|burgers|barbeque|bbq|steak|tacos)"
alcohol_kw = r"(bar|bars|wine|beer|cocktail|brewpub|pub)"
market_kw = r"(market|grocery|supermarket|department stores|shopping)"

# tag presence flags
coffee_biz["has_food"] = coffee_biz["categories_clean"].str.contains(food_kw, regex = True)
coffee_biz["has_alcohol"] = coffee_biz["categories_clean"].str.contains(alcohol_kw, regex = True)
coffee_biz["has_market"] = coffee_biz["categories_clean"].str.contains(market_kw, regex = True)

# concept group (mutually exclusive label)
def concept_group(row):
    if row["has_market"]:
        return "market/retail"
    if row["has_alcohol"]:
        return "coffee+alcohol"
    if row["has_food"]:
        return "food-forward"
    return "pure-coffee"

coffee_biz["concept_group"] = coffee_biz.apply(concept_group, axis=1)
coffee_biz["concept_group"].value_counts()

/var/folders/dm/y0wy_ncs3qn20cs4xdn9sfq00000gn/T/ipykernel_37345/3766344450.py:16: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  coffee_biz["has_food"] = coffee_biz["categories_clean"].str.contains(food_kw, regex = True)
/var/folders/dm/y0wy_ncs3qn20cs4xdn9sfq00000gn/T/ipykernel_37345/3766344450.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  coffee_biz["has_alcohol"] = coffee_biz["categories_clean"].str.contains(alcohol_kw, regex = True)
/var/folders/dm/y0wy_ncs3qn20cs4xdn9sfq00000gn/T/ipykernel_37345/3766344450.py:18: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  coffee_biz["has_market"] = coffee_biz["categories_clean"].str.contains(market_kw, regex = True)


concept_group
food-forward      5530
pure-coffee       1363
coffee+alcohol    1076
market/retail      540
Name: count, dtype: int64

In [22]:
# Save enriched table to SQLite for SQL-first analysis

# keep only useful columns so the DB stays clean
coffee_business_enriched = coffee_biz[[
    "business_id", "name", "address", "city", "state", "postal_code",
    "latitude", "longitude", "stars", "review_count", "is_open", "categories",
    "concept_group", "has_food", "has_alcohol", "has_market"
]].copy()

coffee_business_enriched.to_sql("coffee_business_enriched", conn, if_exists = "replace", index = False)

# sanity check
pd.read_sql_query("""
SELECT concept_group, COUNT(*) AS n_business
FROM coffee_business_enriched
GROUP BY concept_group
ORDER BY n_business DESC;
""", conn)

,concept_group,n_business
0,food-forward,5530
1,pure-coffee,1363
2,coffee+alcohol,1076
3,market/retail,540


## Concept performance - Do some concept mixes earn higher ratings and visuability?

Now that we tagged each coffee-related business into a simple positioning group, we compare outcomes by concept:
- average review stars (from `coffee_reviews`)
- review volume (n_reviews_loaded)
- visibility_score (log-scaled mix of volume + rating)
This helps CoffeeKing decide not only *where* to open, but *what kind of coffee concept* is more likely to succeed.

In [25]:
# SQL: compare concept groups (ratings, volume, visibility)

concept_perf = pd.read_sql_query("""
WITH concept_base AS (
  SELECT 
    b.business_id,
    b.concept_group,
    b.state,
    b.city
  FROM coffee_business_enriched b
),
review_agg AS (
  SELECT 
    business_id,
    COUNT(*) AS n_reviews_loaded,
    AVG(stars) AS avg_review_stars
  FROM coffee_reviews
  GROUP BY business_id
),
scored AS (
  SELECT
    cb.concept_group,
    ra.business_id,
    ra.n_reviews_loaded,
    ra.avg_review_stars,
    (LOG(1 + ra.n_reviews_loaded) * ra.avg_review_stars) AS visibility_score
  FROM review_agg ra
  JOIN concept_base cb
    ON ra.business_id = cb.business_id
)
SELECT
  concept_group,
  COUNT(*) AS n_business,
  ROUND(AVG(avg_review_stars), 3) AS avg_rating,
  ROUND(AVG(n_reviews_loaded), 1) AS avg_reviews,
  ROUND(AVG(visibility_score), 2) AS avg_visibility
FROM scored
GROUP BY concept_group
ORDER BY avg_visibility DESC;
""", conn)

concept_perf


,concept_group,n_business,avg_rating,avg_reviews,avg_visibility
0,coffee+alcohol,1076,4.007,115.5,6.87
1,market/retail,540,4.059,77.2,5.88
2,food-forward,5530,3.470,75.1,5.44
3,pure-coffee,1363,3.575,36.1,4.85


## "Winner rate" - which concept is over-represented among top-visibility businesses?

Instead of only looking at averages (which can be skewed by huge outliers),
we measure how often each concept appears in the **top 10% visiblity** businesses.
This is a simple, business-friendly KPI: "What concepts show up disproportionately among the winners?"

In [26]:
# SQL: top 10% visiblity share by concept (winner rate)

winner_rate = pd.read_sql_query("""
WITH scored AS(
    SELECT
        b.concept_group,
        r.business_id,
        COUNT(*) AS n_reviews_loaded,
        AVG(r.stars) AS avg_review_stars,
        (LOG(1+COUNT(*)) * AVG(r.stars)) AS visibility_score
    FROM coffee_reviews r
    JOIN coffee_business_enriched b
    ON r.business_id = b.business_id
    GROUP BY b.concept_group, r.business_id
),
cutoff AS (
    SELECT
        visibility_score,
        NTILE(10) OVER (ORDER BY visibility_score DESC) AS decile
    FROM scored
),
scored_with_decile AS (
    SELECT
        s.*,
        NTILE(10) OVER (ORDER BY s.visibility_score DESC) AS decile
    FROM scored s
)
SELECT
    concept_group,
    COUNT(*) AS n_business,
    SUM(CASE WHEN decile = 1 THEN 1 ELSE 0 END) AS n_top10pct,
    ROUND(1.0 * SUM(CASE WHEN decile = 1 THEN 1 ELSE 0 END) / COUNT(*), 3) AS top10pct_rate
FROM scored_with_decile
GROUP BY concept_group
ORDER BY top10pct_rate DESC;
""", conn)

winner_rate

,concept_group,n_business,n_top10pct,top10pct_rate
0,coffee+alcohol,1076,189,0.176
1,market/retail,540,59,0.109
2,food-forward,5530,560,0.101
3,pure-coffee,1363,43,0.032


### Interpretation (Concept performance)

- Overall, **coffee+alcohol** stands out as the strongest "visibility" concept:
    - highest avg_visibility (6.87)
    - high avg_rating (4.007) and the highest avg_reviews (115.5)
    - also dominates the "winner rate" metric: **17.6%** of coffee+alcohol businesses land in the top 10% visibility.

- **market/retail** is the most consistently high-rated group (avg_rating 4.059) with solid visibility (5.88), suggesting this concept may perform well when paired with a strong retail/market destination angle.

- **food-forward** is the largest bucket (5,530 businesses) but has the lowest avg_rating (3.470).
Its winner rate (10.1%) is close to market/retail (10.9), which suggests a small subset of food-forward "destination" places can still break out, but on average this concept is more competitive / mixed-quality.

- **pure-coffee** is the weakest on visibility:
    - lowest avg_reviews (36.1) and lowest avg_visibility (4.85)
    - winner rate is only **3.2%**, meaning it is under-represented among top performers. 

CoffeeKing takeaway:
If CoffeeKing's goal is to maximize early discoverability and demand, the data suggests prioritizing a **coffee+alcohol** hybrid (or a concept that boosts "evening + social" usage), instead of launching as a pure coffee shop. A safer "premium" alternative is **market/retail**, which shows strong ratings with decent visibility. 

### Location + Concept "Winner Rate" (CoffeeKing Recommendation Engine)

Goal:
Identify the best **city/state + concept_group** combinations for CoffeeKing by measuring "winner rate".

Definition:
- We already defined a **top performer** as businesses in the **top 10% of visibility_score**.
- Here, we compute winner rate by location and concept:
winner_rate = (# top10% businesses) / (total businesses)

Output:
Top 10 location + concept combos with:
- enough sample size (to avoid tiny cities dominating)
- high winner_rate
- plus helpful context (avg_rating, avg_reviews, avg_visibility)

In [ ]:
base_scored = pd.read_sql_query("""
WITH scored AS (
  SELECT
    b.business_id,
    b.state,
    b.city,
    b.concept_group,
    COUNT(r.review_id) AS n_reviews_loaded,
    AVG(r.stars) AS avg_review_stars,
    (LOG(1 + COUNT(r.review_id)) * AVG(r.stars)) AS visibility_score
  FROM coffee_reviews r
  JOIN coffee_business_enriched b
    ON r.business_id = b.business_id
  GROUP BY b.business_id, b.state, b.city, b.concept_group
),
scored_with_decile AS (
  SELECT
    *,
    NTILE(10) OVER (ORDER BY visibility_score DESC) AS decile
  FROM scored
)
SELECT * FROM scored_with_decile;
""", conn)

base_scored.shape, base_scored.head(3)


((8509, 8),
               business_id state              city  concept_group  \
 0  ytynqOUb3hjKeJfRj5Tshw    PA      Philadelphia  market/retail   
 1  oBNrLz4EDhiscSlbOl8uAw    LA       New Orleans   food-forward   
 2  2KIDQyTh-HzLxOUEDqtDBg    FL  Saint Petersburg  market/retail   
 
    n_reviews_loaded  avg_review_stars  visibility_score  decile  
 0              5778          4.605400         17.324836       1  
 1              5264          4.291223         15.969352       1  
 2              1583          4.641188         14.850664       1  )

In [49]:
MIN_N = 25  

top10_location_concept = pd.read_sql_query(f"""
WITH scored AS (
  SELECT
    b.business_id,
    b.state,
    b.city,
    b.concept_group,
    COUNT(r.review_id) AS n_reviews_loaded,
    AVG(r.stars) AS avg_review_stars,
    (LOG(1 + COUNT(r.review_id)) * AVG(r.stars)) AS visibility_score
  FROM coffee_reviews r
  JOIN coffee_business_enriched b
    ON r.business_id = b.business_id
  GROUP BY b.business_id, b.state, b.city, b.concept_group
),
scored_with_decile AS (
  SELECT
    *,
    NTILE(10) OVER (ORDER BY visibility_score DESC) AS decile
  FROM scored
),
combo AS (
  SELECT
    state,
    city,
    concept_group,
    COUNT(*) AS n_business,
    SUM(CASE WHEN decile = 1 THEN 1 ELSE 0 END) AS n_top10pct,
    ROUND(1.0 * SUM(CASE WHEN decile = 1 THEN 1 ELSE 0 END) / COUNT(*), 3) AS winner_rate,
    ROUND(AVG(avg_review_stars), 3) AS avg_rating,
    ROUND(AVG(n_reviews_loaded), 1) AS avg_reviews,
    ROUND(AVG(visibility_score), 2) AS avg_visibility
  FROM scored_with_decile
  GROUP BY state, city, concept_group
)
SELECT *
FROM combo
WHERE n_business >= {MIN_N}
ORDER BY winner_rate DESC, avg_visibility DESC
LIMIT 10;
""", conn)

top10_location_concept


,state,city,concept_group,n_business,n_top10pct,winner_rate,avg_rating,avg_reviews,avg_visibility
0,CA,Santa Barbara,coffee+alcohol,27,11,0.407,4.107,235.7,8.99
1,LA,New Orleans,coffee+alcohol,73,24,0.329,4.015,227.5,7.84
2,MO,Saint Louis,coffee+alcohol,34,11,0.324,4.059,161.0,7.88
3,IN,Indianapolis,coffee+alcohol,41,12,0.293,4.198,172.3,7.90
4,CA,Santa Barbara,food-forward,85,23,0.271,3.810,179.6,7.33
5,LA,New Orleans,food-forward,257,66,0.257,3.783,209.8,7.04
6,NV,Reno,market/retail,26,6,0.231,3.630,67.1,5.88
7,FL,Tampa,coffee+alcohol,92,21,0.228,4.016,117.9,7.15
8,NV,Reno,coffee+alcohol,43,9,0.209,4.005,155.6,7.66
9,CA,Goleta,food-forward,30,6,0.200,3.853,115.5,6.94


In [52]:
MIN_CITY_N = 80

top_cities = pd.read_sql_query(f"""
WITH scored AS (
  SELECT
    b.business_id,
    b.state,
    b.city,
    COUNT(r.review_id) AS n_reviews_loaded,
    AVG(r.stars) AS avg_review_stars,
    (LOG(1 + COUNT(r.review_id)) * AVG(r.stars)) AS visibility_score
  FROM coffee_reviews r
  JOIN coffee_business_enriched b
    ON r.business_id = b.business_id
  GROUP BY b.business_id, b.state, b.city
),
scored_with_decile AS (
  SELECT
    *,
    NTILE(10) OVER (ORDER BY visibility_score DESC) AS decile
  FROM scored
),
city AS (
  SELECT
    state,
    city,
    COUNT(*) AS n_business,
    SUM(CASE WHEN decile = 1 THEN 1 ELSE 0 END) AS n_top10pct,
    ROUND(1.0 * SUM(CASE WHEN decile = 1 THEN 1 ELSE 0 END) / COUNT(*), 3) AS city_winner_rate,
    ROUND(AVG(avg_review_stars), 3) AS avg_rating,
    ROUND(AVG(n_reviews_loaded), 1) AS avg_reviews,
    ROUND(AVG(visibility_score), 2) AS avg_visibility
  FROM scored_with_decile
  GROUP BY state, city
)
SELECT *
FROM city
WHERE n_business >= {MIN_CITY_N}
ORDER BY city_winner_rate DESC, avg_visibility DESC
LIMIT 10;
""", conn)

top_cities


,state,city,n_business,n_top10pct,city_winner_rate,avg_rating,avg_reviews,avg_visibility
0,CA,Santa Barbara,160,44,0.275,3.857,173.2,7.47
1,LA,New Orleans,422,95,0.225,3.848,185.6,6.99
2,MO,Saint Louis,197,41,0.208,3.872,112.6,6.58
3,FL,Saint Petersburg,90,16,0.178,3.835,108.5,6.63
4,NV,Reno,265,41,0.155,3.658,103.6,6.24
5,IN,Indianapolis,375,56,0.149,3.602,83.6,5.84
6,FL,Clearwater,110,16,0.145,3.539,62.3,5.54
7,TN,Nashville,394,56,0.142,3.697,98.3,6.24
8,PA,Philadelphia,1066,147,0.138,3.597,100.4,5.89
9,AZ,Tucson,395,53,0.134,3.601,82.6,5.87


In [53]:
winner_city_pairs = [(row["state"], row["city"]) for _, row in top_cities.iterrows()]
city_filter_sql = ", ".join([f"('{s}','{c}')" for s, c in winner_city_pairs])

city_concept_compare = pd.read_sql_query(f"""
WITH scored AS (
  SELECT
    b.business_id,
    b.state,
    b.city,
    b.concept_group,
    COUNT(r.review_id) AS n_reviews_loaded,
    AVG(r.stars) AS avg_review_stars,
    (LOG(1 + COUNT(r.review_id)) * AVG(r.stars)) AS visibility_score
  FROM coffee_reviews r
  JOIN coffee_business_enriched b
    ON r.business_id = b.business_id
  GROUP BY b.business_id, b.state, b.city, b.concept_group
),
scored_with_decile AS (
  SELECT
    *,
    NTILE(10) OVER (ORDER BY visibility_score DESC) AS decile
  FROM scored
),
combo AS (
  SELECT
    state,
    city,
    concept_group,
    COUNT(*) AS n_business,
    SUM(CASE WHEN decile = 1 THEN 1 ELSE 0 END) AS n_top10pct,
    ROUND(1.0 * SUM(CASE WHEN decile = 1 THEN 1 ELSE 0 END) / COUNT(*), 3) AS winner_rate,
    ROUND(AVG(avg_review_stars), 3) AS avg_rating,
    ROUND(AVG(n_reviews_loaded), 1) AS avg_reviews,
    ROUND(AVG(visibility_score), 2) AS avg_visibility
  FROM scored_with_decile
  GROUP BY state, city, concept_group
)
SELECT *
FROM combo
WHERE (state, city) IN ({city_filter_sql})
  AND concept_group IN ('coffee+alcohol', 'market/retail')
ORDER BY state, city, winner_rate DESC;
""", conn)

city_concept_compare


,state,city,concept_group,n_business,n_top10pct,winner_rate,avg_rating,avg_reviews,avg_visibility
0,AZ,Tucson,coffee+alcohol,62,10,0.161,4.035,122.6,7.27
1,AZ,Tucson,market/retail,22,3,0.136,3.759,83.7,5.52
2,CA,Santa Barbara,coffee+alcohol,27,11,0.407,4.107,235.7,8.99
3,CA,Santa Barbara,market/retail,17,6,0.353,4.039,153.1,7.67
4,FL,Clearwater,market/retail,3,1,0.333,3.883,69.3,5.96
5,FL,Clearwater,coffee+alcohol,13,3,0.231,3.904,91.0,6.32
6,FL,Saint Petersburg,coffee+alcohol,19,5,0.263,4.199,170.0,7.77
7,FL,Saint Petersburg,market/retail,10,2,0.200,4.174,180.4,5.94
8,IN,Indianapolis,coffee+alcohol,41,12,0.293,4.198,172.3,7.90
9,IN,Indianapolis,market/retail,27,4,0.148,4.227,69.2,6.34


## Location + Concept "Winner Rate" (CoffeeKing Recommendation Engine)

**Goal**: Identify the best city + concept combinations for CoffeeKing by measuring how often businesses become "winners" (top performers) within each location.

**Definitions**
- **Visibility Score** = log (1 + n_reviews_loaded) x avg_review_stars (captures "well-known AND well-liked" while reducing review-count outlier effects)
- **Winner (Top 10%)** = businesses in the top decile of visibility_score
- **Winner rate (by city/concept)** = (# winners) / (total businesses) within that city + concept_group 

**Key Findings (City-level)**

The strongest "winner cities" by city_winner_rate were:
    - Santa Barabara, CA (0.275)
    - New Orleans, LA (0.225)
    - Saint Louis, MO (0.208)
    - Saint Petersburg, FL (0.178)
Large cities like Philadelphia appear frequently among top businesses, but their city-level winner rate is lower--suggesting scale produces many winners, but not the strongest probability of winning. 

**Key findings (Concept within winner cities)**

Across most top cities, *coffee+alcohol* has a higher winner rate than market/retail, indicating that a coffee concept with an evening/social angle is disproportionately represented among top performers. One notable exception is *Reno, NV*, where market/retail slightly outperforms coffee+alcohol - suggesting retail/market positioning may be a stronger local fit there.

### CoffeeKing "Top Combo" recommendations (Winner rate leaers)

- Santa Barbara, CA - coffee+alcohol (winner_rate 0.407)
- Santa Barbara, CA - market/retail (0.353)
- New Orleans, LA - coffee+alcohol (0.329)
- Saint Louis, MO - coffee+alcohol (0.324)
- Indianapolis, IN - coffee+alcohol (0.293)
- Saint Petersburg, FL - coffee+alcohol (0.263)
- Reno, NV - market/retail (0.231)

### Actionable Takeaway

If CoffeeKing's priority is fast discoverability + strong ratings, the data supports launching a coffee+alcohol hybrid in top winner cities (e.g., Santa Barbara, New Orleans), while treating market/retail as a strong secondary concept - especially in markets like Reno where performs competitively. *(When interpreting very small sample sizes for a city/concept combo, we should treat those as "signlas" rather than definitive rankings.)*

## 9. CoffeeKing Recommendation Engine v1

This project is not just exploratory analysis - it produces an actionable expansion shortlist for a hypothetical brand, CoffeeKing.

**Core Idea**
- Visibility Score = log(1+review_volume) x average review stars
- Winners = top 10% of businesses by visibility score
- Winner rate = (# winners) / (# businesses) within a city or a cityxconcept segment

**Deliverables produced**
1) Top cities where "winning" is more likely (city-level winner rate, with minimum sample size filter)
2) Top city x concept combinations (CoffeeKing launch "playbook")
3) A saved SQLite table (`coffeeking_reco_v1`) so the results can be reused without re-running the whole notebook.

In [55]:
# Build final recommendation table (CoffeeKing Reco v1) 

# 1) City-level ranking (top cities)
cities_out = top_cities.copy()
cities_out["level"] = "city"
cities_out["concept_group"] = None
cities_out = cities_out.rename(columns={"city_winner_rate": "winner_rate"})

# 2) City x concept ranking (top combos)
combo_out = top10_location_concept.copy()
combo_out["level"] = "city_concept"

# 3) Focus comparison (coffee+alcohol vs market/retail in winner cities)
compare_out = city_concept_compare.copy()
compare_out["level"] = "winner_city_concept_compare"

# Make columns consistent
def align_cols(df):
    keep = ["level", "state", "city", "concept_group",
            "n_business", "n_top10pct", "winner_rate",
            "avg_rating", "avg_reviews", "avg_visibility"]
    for c in keep:
        if c not in df.columns:
            df[c] = None
    return df[keep]

final_reco = pd.concat([
    align_cols(cities_out),
    align_cols(combo_out),
    align_cols(compare_out)
], ignore_index = True)

final_reco.head()


,level,state,city,concept_group,n_business,n_top10pct,winner_rate,avg_rating,avg_reviews,avg_visibility
0,city,CA,Santa Barbara,None,160,44,0.275,3.857,173.2,7.47
1,city,LA,New Orleans,None,422,95,0.225,3.848,185.6,6.99
2,city,MO,Saint Louis,None,197,41,0.208,3.872,112.6,6.58
3,city,FL,Saint Petersburg,None,90,16,0.178,3.835,108.5,6.63
4,city,NV,Reno,None,265,41,0.155,3.658,103.6,6.24


In [ ]:
final_reco.to_sql("coffeeking_reco_v1", conn, if_exists="replace", index=False)

pd.read_sql_query("SELECT level, COUNT(*) AS n_rows FROM coffeeking_reco_v1 GROUP BY level;", conn)

,level,n_rows
0,city,10
1,city_concept,10
2,winner_city_concept_compare,20


# 10. CoffeeKing Launch Playbook (How to use these results)

### Step A - Pick expansion markets (city shortlist)

Use `city_winner_rate` to shortlist cities where a new store has a higher probability of becoming highly visible.

**Current top markets (min city sample filter applied):**

- Santa Barbara, CA
- New Orleans, LA
- Saint Louis, MO
- Saint Petersburg, FL
- Reno, NV

### Step B - Pick the concept mix (concept fit within market)

In most top cities, **coffee+alcohol** has a higher winner rate than **market/retail**, suggesting "evening + social" positioning is a stronger visibility strategy.
**Exception:** Reno, NV where market/retail performs slightly better.

### Step C - Risk Control (sample size rule)

City x concept rankings can be distorted by small sample sizes, so we used minimum thresholds:
- MIN_CITY_N = 80 for city shortlist
- MIN_N = 25 for cityxconcept combos

Interpret high winner rate below the threshold as *signals*, not final decisions.

### Final Recommendation (Pilot)

Pilot in:
1) Santa Barbara, CA - coffee+alcohol
2) New Orleans, LA - coffee+alcohol

Keep as alternatives:
- Santa Barbara - market/retail
- Reno - market/retail

